# Instalando Bibliotecas


In [ ]:
!pip install geocoder
import geocoder
!pip install haversine
from haversine import haversine, Unit
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor
from polyglot.downloader import downloader
from polyglot.text import Text
!polyglot download ner2.pt
!polyglot download ner2.en
import pandas as pd
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import numpy as np
import networkx as nx
!pip install plotly.express
from plotly import graph_objs as go
import folium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52561 sha256=7e8d5f6960d1c85f70ceee63f3e4c169b389a10f480313a238b49717ff957577
  Stored in directory: /root/.cache/pip/wheels/aa/92/4a/b172589446ba537db3bdb9a1f2204f27fe71217981c14ac368
Successfully built polyglot
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyicu: filename=PyICU-2.13.1-cp310-cp310-linux_x86_64.whl size=1801407 sha256=455e5459f65ddc94e686d6e1c3766d877e894daa37af96a8e71b039038e62323
  Stored in directory: /root/.cache/pip/wheels/8e/45/08/f4bc505e31eb7bb779d493226921abf18cf55ed30142e70eae
Successfully built pyicu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 27.9 MB/s et

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [ ]:
!polyglot download embeddings2.pt

[polyglot_data] Downloading package embeddings2.pt to
[polyglot_data]     /root/polyglot_data...


# Extraindo Entidades Nomeadas

In [ ]:
# identificando entidades nomeadas em textos
text = Text("O curso de mineração de dados não estruturados na USP de São Carlos - SP está ocorrendo hoje")
text.entities

[I-ORG(['USP']), I-LOC(['São', 'Carlos']), I-LOC(['SP'])]

# Filtrando Entidades Nomeadas sobre Informação Geográfica

In [ ]:
# extraindo entidades geográficas (tipo I-LOC)
text = Text("A escola avançada de big data na USP de São Carlos - SP está ocorrendo hoje")
loc = ""
for ent in text.entities:
  if ent.tag=='I-LOC':
    for s in ent:
      loc += s+" "
print(loc)

São Carlos SP 


# Aplicando Geocoder (identificando lat/lng)

In [ ]:
# extraindo latitude / longitude
print("Geocoding loc: "+loc)
g = geocoder.geonames(loc, maxRows=1, key='3eabda_1')
for result in g:
  print("Country: "+result.country)
  print("State: "+result.state)
  print("Address: "+result.address)
  print("Lat: "+result.lat)
  print("Lng: "+result.lng)

Geocoding loc: São Carlos SP 
Country: Brazil
State: São Paulo
Address: São Carlos
Lat: -22.0175
Lng: -47.89083


In [ ]:
# extraindo latitude / longitude
loc = 'Tokyo, Japan'
print("Geocoding loc: "+loc)
g = geocoder.geonames(loc, maxRows=1, key='3eabda_1')
for result in g:
  print("Country: "+result.country)
  print("State: "+result.state)
  print("Address: "+result.address)
  print("Lat: "+result.lat)
  print("Lng: "+result.lng)

Geocoding loc: Tokyo, Japan
Country: Japan
State: Tokyo
Address: Tokyo
Lat: 35.6895
Lng: 139.69171


# Calculando distância entre localizações (usando Haversine)

In [ ]:
# calculando distancia haversine entre duas localidades
loc1 = "São Carlos, SP"
loc2 = "Tokyo, Japan"
g1 = geocoder.geonames(loc1, maxRows=1, key='3eabda_1')
g2 = geocoder.geonames(loc2, maxRows=1, key='3eabda_1')
dist_km = haversine( (float(g1[0].lat),float(g1[0].lng)), (float(g2[0].lat),float(g2[0].lng)))
print("Dist (km) = "+str(dist_km))

Dist (km) = 18326.51097643297


In [ ]:
# calculando distancia haversine entre duas localidades
loc1 = "São Carlos, SP"
loc2 = "Araraquara, SP"
g1 = geocoder.geonames(loc1, maxRows=1, key='3eabda_1')
g2 = geocoder.geonames(loc2, maxRows=1, key='3eabda_1')
dist_km = haversine( (float(g1[0].lat),float(g1[0].lng)), (float(g2[0].lat),float(g2[0].lng)))
print("Dist (km) = "+str(dist_km))

Dist (km) = 38.44558054334914


## Base Textual

In [ ]:
import urllib.parse
# obtendo dataset com uma amostra de textos

query = "febre amarela" # query para consultar
query = urllib.parse.quote(query)

url = 'http://websensors.net.br/minicurso/2019/eventos-br-2017.php?q='+query
print(url)

http://websensors.net.br/minicurso/2019/eventos-br-2017.php?q=febre%20amarela


In [ ]:
!wget '{url}' -O dados.csv

--2023-08-25 08:35:37--  http://websensors.net.br/minicurso/2019/eventos-br-2017.php?q=febre%20amarela
Resolving websensors.net.br (websensors.net.br)... 129.121.3.177
Connecting to websensors.net.br (websensors.net.br)|129.121.3.177|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘dados.csv’

dados.csv               [         <=>        ]   2.84M  1.75MB/s    in 1.6s    

2023-08-25 08:35:39 (1.75 MB/s) - ‘dados.csv’ saved [2979374]



In [ ]:
import pandas as pd
dataset = pd.read_csv('dados.csv',sep='\t', lineterminator='\n').sample(1000)
dataset

,event_id,date,title,local,url,lat,lng
8405,6a6b423a36d7e5119edd35bb3b999667,2017-12-14 15:59:34,Febre amarela já matou dez pessoas e 501 macac...,Sao Paulo (SP),http://news.google.com/news/url?sa=t&amp;fd=R&...,-23.54,-46.63
798,349b55aaf554271e306c753b3bd69d16,2017-01-21 10:56:21,SP apura 4 mortes por febre amarela - Istoe,Registro (SP),http://istoe.com.br/sp-apura-4-mortes-por-febr...,-24.48,-47.84
7954,3274c39a3c6d792fd9ba183478ac01b6,2017-11-06 20:21:14,Febre amarela causa a morte de quase 90 macaco...,Campo Limpo Paulista (SP),https://g1.globo.com/sao-paulo/sorocaba-jundia...,-23.20,-46.78
3127,1b5be0be4df87c2651f887cced899094,2017-02-21 10:05:39,Febre amarela leva União a reconhecer situação...,Canoas (RS),http://news.google.com/news/url?sa=t&amp;fd=R&...,-29.91,-51.18
5230,5708724f0eafbad00fc2f4ef24a4c465,2017-04-06 15:16:00,Rio: recomendação de dose única deve reduzir c...,Rio de Janeiro (RJ),http://news.google.com/news/url?sa=t&amp;fd=R&...,-22.90,-43.20
...,...,...,...,...,...,...,...
1133,146cd8b04770b4ef932e96b47ac96324,2017-01-24 23:12:00,Secretário e prefeitos debatem ações de combat...,Sao Paulo (SP),http://atarde.uol.com.br/bahia/noticias/183357...,-23.54,-46.63
3954,d52855903b2376a7922d75a4287cd1a3,2017-03-11 10:12:33,Minas tem 55 cidades com risco para febre amar...,Timoteo (MG),http://www.em.com.br/app/noticia/gerais/2017/0...,-19.58,-42.64
7678,395ad30674200f245530fad443299c8c,2017-10-25 12:15:04,Sergipe tem 12 mil vacinas contra febre amarel...,Sao Cristovao (SE),https://g1.globo.com/se/sergipe/noticia/sergip...,-11.01,-37.20
2364,9c34771e8b7a211d8a201880770bceda,2017-01-10 07:59:13,Surto de febre amarela provoca corrida por vac...,Monte Alto (SP),http://www.em.com.br/app/noticia/gerais/2017/0...,-21.26,-48.49


In [ ]:
s  = ''
for index,row in dataset.iterrows():

  s += '''
  {
    name: "'''+row['local']+'''",
    url: 'http://labic.icmc.usp.br',
    lat: '''+str(row['lat'])+''',
    lng: '''+str(row['lng'])+''',
  },
  '''

In [ ]:
with open('makers.txt', 'w') as f:
    f.write(s)

In [ ]:
# filtrando dados de interesse
dataset = dataset[['title','local','lat','lng']]
dataset

,title,local,lat,lng
8405,Febre amarela já matou dez pessoas e 501 macac...,Sao Paulo (SP),-23.54,-46.63
798,SP apura 4 mortes por febre amarela - Istoe,Registro (SP),-24.48,-47.84
7954,Febre amarela causa a morte de quase 90 macaco...,Campo Limpo Paulista (SP),-23.20,-46.78
3127,Febre amarela leva União a reconhecer situação...,Canoas (RS),-29.91,-51.18
5230,Rio: recomendação de dose única deve reduzir c...,Rio de Janeiro (RJ),-22.90,-43.20
...,...,...,...,...
1133,Secretário e prefeitos debatem ações de combat...,Sao Paulo (SP),-23.54,-46.63
3954,Minas tem 55 cidades com risco para febre amar...,Timoteo (MG),-19.58,-42.64
7678,Sergipe tem 12 mil vacinas contra febre amarel...,Sao Cristovao (SE),-11.01,-37.20
2364,Surto de febre amarela provoca corrida por vac...,Monte Alto (SP),-21.26,-48.49


# Plotando a informação geográfica dos textos

In [ ]:
# plotando eventos no mapa
m = folium.Map([-14.5931291,-56.6985808], zoom_start=4)

for index,row in dataset.iterrows():
  lat = row['lat']
  lng = row['lng']
  text = row['title']
  folium.Marker([lat, lng], popup=text).add_to(m)

m

In [ ]:
# plotando eventos no mapa com heatmap
from folium.plugins import HeatMap

m = folium.Map([-14.5931291,-56.6985808], zoom_start=4)
data_heatmap = []
for index,row in dataset.iterrows():
  lat = row['lat']
  lng = row['lng']
  text = row['title']
  data_heatmap.append([lat,lng,1])

HeatMap(data_heatmap).add_to(m)

m